SQL形式のデータからClient.csvとRecord.csvを抽出する

In [ ]:
import sqlite3
import csv

from google.colab import drive
drive.mount('/content/drive')

dbpath = '/' 
# データベースのパスを指定するように適宜変更する

conn = sqlite3.connect(dbpath)

# カーソルの取得
cur = conn.cursor()

# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

# Clientテーブルから全データを取得
cur.execute('select * from Client')
with open('Client.csv','w',newline = '') as csv_file:
  csv_writer = csv.writer(csv_file)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

# Recordテーブルから全データを取得
cur.execute('select * from Record')
with open('Record.csv','w',newline = '') as csv_file:
  csv_writer = csv.writer(csv_file)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

# 処理の確定
conn.commit()

# カーソルを閉じる
cur.close()

# 接続を閉じる
conn.close()

ライブラリとデータセットを導入する

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

Client = pd.read_csv("/content/Client.csv")
Record = pd.read_csv("/content/Record.csv")

from google.colab import drive
drive.mount('/content/drive')

print('データのデータ数は{}、変数は{}種類です。'.format(Client.shape[0], Client.shape[1]))
print('データのデータ数は{}、変数は{}種類です。'.format(Record.shape[0], Record.shape[1]))

データセットの確認を行う

In [ ]:
print(Client.info())

In [ ]:
print(Record.info())

In [ ]:
Client.head()

In [ ]:
Record.head()

欠損値の確認と補完を行う

In [ ]:
Client.isnull().sum()

In [ ]:
Record.isnull().sum()

In [ ]:
# Recordの欠損値を平均値で補完
Record["rev_Mean"].fillna(Record["rev_Mean"].mean(), inplace=True)
Record["mou_Mean"].fillna(Record["mou_Mean"].mean(), inplace=True)
Record["totmrc_Mean"].fillna(Record["totmrc_Mean"].mean(), inplace=True)
Record["da_Mean"].fillna(Record["da_Mean"].mean(), inplace=True)
Record["ovrmou_Mean"].fillna(Record["ovrmou_Mean"].mean(), inplace=True)
Record["ovrrev_Mean"].fillna(Record["ovrrev_Mean"].mean(), inplace=True)
Record["vceovr_Mean"].fillna(Record["vceovr_Mean"].mean(), inplace=True)
Record["datovr_Mean"].fillna(Record["datovr_Mean"].mean(), inplace=True)
Record["roam_Mean"].fillna(Record["roam_Mean"].mean(), inplace=True)
Record["change_mou"].fillna(Record["change_mou"].mean(), inplace=True)
Record["change_rev"].fillna(Record["change_rev"].mean(), inplace=True)

In [ ]:
Record.isnull().sum()

各変数の可視化を行う

In [ ]:
Record['rev_Mean'].sum()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
Record['churn'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('churn')
ax[0].set_ylabel('')
sns.countplot('churn',data=Record,ax=ax[1])
ax[1].set_title('churn')
plt.show()

In [ ]:
f,ax=plt.subplots(1,1,figsize=(18,8))
Labels=['','']
Record['churn'].value_counts().plot.pie(explode=[0,0.1],labels=Labels,shadow=True)
plt.title('')
plt.ylabel('')

In [ ]:
pip install japanize-matplotlib

In [ ]:
Record['churn'].value_counts()

In [ ]:
import japanize_matplotlib

x = [50438, 49562]
Labels = ['非解約者', '解約者']
plt.pie(x, labels=Labels, startangle=90, counterclock=False, autopct='%1.1f%%', shadow=True)
plt.title('解約したかどうか')

LightGBM

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
X = Record.drop('churn',axis=1) 
y = Record['churn'] # 目的変数
# トレーニングデータ,テストデータの分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2, random_state=0)

In [ ]:
# 学習に使用するデータを設定
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train) 

# パラメータ
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary', # 目的 : 分類
        'metric': {'auc'},
         'num_leaves': 20,             
        'max_depth':5,
        'min_data_in_leaf': 3,        
        'num_iteration': 100,            
        'learning_rate':0.03,
        'num_boost_round':100,
         'early_stopping_rounds':20, 
}

# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

# テストデータの予測
y_pred = model.predict(X_valid)

In [ ]:
lgb.plot_importance(model, height=0.5, figsize=(30,40))

解約率に影響を与えると考えられる変数を探索する

In [ ]:
Record.groupby(["churn"])["change_mou"].mean()

In [ ]:
Record.groupby(["churn"])["change_mou"].median()

In [ ]:
sns.barplot(x='churn', y='change_mou', data=Record, estimator=np.median)

In [ ]:
Record.groupby('churn')['change_mou'].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
from scipy import stats
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["change_mou"]
                , Record[Record["churn"]==0]["change_mou"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["months"].mean()

In [ ]:
Record.groupby(["churn"])["months"].median()

In [ ]:
sns.barplot(x='churn', y='months', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["months"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["months"]
                , Record[Record["churn"]==0]["months"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["mou_Mean"].mean()

In [ ]:
Record.groupby(["churn"])["mou_Mean"].median()

In [ ]:
sns.barplot(x='churn', y='mou_Mean', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["mou_Mean"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["mou_Mean"]
                , Record[Record["churn"]==0]["mou_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["totmrc_Mean"].mean()

In [ ]:
Record.groupby(["churn"])["totmrc_Mean"].median()

In [ ]:
sns.barplot(x='churn', y='totmrc_Mean', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["totmrc_Mean"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["totmrc_Mean"]
                , Record[Record["churn"]==0]["totmrc_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["change_rev"].mean()

In [ ]:
Record.groupby(["churn"])["change_rev"].median()

In [ ]:
sns.barplot(x='churn', y='change_rev', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["change_rev"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["change_rev"]
                , Record[Record["churn"]==0]["change_rev"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["vceovr_Mean"].mean()

In [ ]:
Record.groupby(["churn"])["vceovr_Mean"].median()

In [ ]:
sns.barplot(x='churn', y='vceovr_Mean', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["vceovr_Mean"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["vceovr_Mean"]
                , Record[Record["churn"]==0]["vceovr_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
Record.groupby(["churn"])["rev_Mean"].mean()

In [ ]:
Record.groupby(["churn"])["rev_Mean"].median()

In [ ]:
sns.barplot(x='churn', y='rev_Mean', data=Record, estimator=np.median)

In [ ]:
Record.groupby(["churn"])["rev_Mean"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["rev_Mean"]
                , Record[Record["churn"]==0]["rev_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
sns.barplot(x='churn', y='ovrrev_Mean', data=Record, estimator=np.median)

In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["ovrrev_Mean"]
                , Record[Record["churn"]==0]["ovrrev_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
sns.barplot(x='churn', y='ovrmou_Mean', data=Record, estimator=np.median)


In [ ]:
s, pvalue = stats.mannwhitneyu(Record[Record["churn"]==1]["ovrrev_Mean"]
                , Record[Record["churn"]==0]["ovrrev_Mean"]
                ,alternative='two-sided')
pvalue < 0.05

In [ ]:
# ライブラリの読み込み
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# 目的変数と説明変数に分割
X = Record[["change_mou", "mou_Mean", "totmrc_Mean", "rev_Mean", "change_rev", "ovrmou_Mean", "ovrrev_Mean", "months"]].values
y = Record["churn"].values

# 標準化
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

# 訓練データと評価データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_std, y, test_size=0.3, stratify=y, random_state=0)

In [ ]:
# LGBMClassifier
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=0)
lgb.fit(X_train, y_train)

lgb_train_pred = lgb.predict_proba(X_train)[:, 1]
lgb_valid_pred = lgb.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, lgb_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgb_valid_pred)}")

In [ ]:
# テストデータに対する予測値の作成
pred = lgb.predict(X_valid)
pred

In [ ]:
cust = pd.DataFrame()
cust['churn'] = pred
cust['churn']
cust[cust['churn']==1]

In [ ]:
f,ax=plt.subplots(1,1,figsize=(18,8))
cust['churn'].value_counts().plot.pie(explode=[0,0.1],labels=Labels,shadow=True)
plt.title('')
plt.ylabel('')

In [ ]:
Record['now_charge'] = Record['rev_Mean'] - Record['ovrrev_Mean']
Record['now_ovr_charge'] = Record['ovrrev_Mean']/Record['ovrmou_Mean']
Record['nor_time'] = Record['mou_Mean'] - Record['ovrmou_Mean']
Record['res_totmrc'] = Record['totmrc_Mean'] - Record['rev_Mean'] - Record['ovrrev_Mean']
Record['now_charge'].median()

In [ ]:
Record['now_ovr_charge'].median()

In [ ]:
Record['nor_time'].median()

In [ ]:
Record['new_charge'] = Record['now_charge'] - 3
Record['new_ovr_charge'] = Record['now_ovr_charge'] - 0.15

In [ ]:
Record['ovrrev_Mean'] = Record['new_ovr_charge']*Record['ovrmou_Mean']
Record['rev_Mean'] = Record['new_charge'] + Record['ovrrev_Mean']
Record['totmrc_Mean'] = Record['res_totmrc'] + Record['new_charge'] + Record['ovrrev_Mean']

In [ ]:
X = Record[["change_mou", "mou_Mean", "totmrc_Mean", "rev_Mean", "change_rev", "ovrmou_Mean", "ovrrev_Mean", "months"]].values
y = Record["churn"].values

# 訓練データと評価データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [ ]:
# テストデータに対する予測値の作成
pred = lgb.predict(X_valid)
pred

In [ ]:
cust = pd.DataFrame()
cust['churn'] = pred
cust['churn']
cust[cust['churn']==1]

In [ ]:
f,ax=plt.subplots(1,1,figsize=(18,8))
cust['churn'].value_counts().plot.pie(explode=[0,0.1],labels=Labels,shadow=True)
plt.title('')
plt.ylabel('')

In [ ]:
Record[['now_charge', 'now_ovr_charge', 'nor_time']].describe()

In [ ]:
df = pd.DataFrame()
df['now_ovr_charge'] = Record['now_ovr_charge']
df['ovrmou_Mean'] = Record['ovrmou_Mean']
df['ovrrev_Mean'] = Record['ovrrev_Mean']
df.dropna().describe()

In [ ]:
Record['rev_Mean'].sum()

In [ ]:
Record[['charge1', 'ovr_charge', 'new_charge', 'new_ovr_charge']].head()

In [ ]:
Record[['new_ovr_rev', 'ovrrev_Mean']].head()

In [ ]:
Record['new_rev_Mean'] = Record['new_charge'] + Record['new_ovr_rev']
Record['new_rev_Mean'].sum()

In [ ]:
loss_sum = Record[Record["churn"]==1]
loss_sum['rev_Mean'].sum()
get_sum = Record[Record["churn"]==0]
get_sum['rev_Mean'].sum()
Record['rev_Mean'].sum()
get_sum['ovrrev_Mean'].sum()
loss_sum['ovrrev_Mean'].sum()
Record['ovrrev_Mean'].sum()